In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
def get_df(bot1, bot2):
    return pd.read_csv('./' + bot1 + '_vs_' + bot2 + '.csv')

In [18]:
aggregations = {
    "appleEaten0" : [np.mean, np.sum],
    "appleEaten1" : [np.mean, np.sum],
    "duration" : [np.mean, np.sum],
    "winner" : ["count"],
}

In [19]:
def get_bots_playing(df):
    bots = list(df["winner"].unique())
    try:
        bots.remove("tie")
    except:
        pass
    bot1, bot2 = bots

    return bot1, bot2

In [31]:
def add_score_columns(df):
    """
        create a tournament score for each bot
        win:  3 points
        tie:  1 point
        loss: 0 points

        returns (points: bot 1), (points: bot 2), (games played) 
    """

    bot1, bot2 = get_bots_playing(df)
    
    bot1_points = 0
    bot2_points = 0
    for index, row in df.iterrows():

        if row["winner"] == bot1:
            bot1_points += 1
            bot2_points -= 1
        elif row["winner"] == bot2:
            bot2_points += 1
            bot1_points -= 1
        

    return bot1_points / (len(df) - len(df[df["winner"] == "tie"])), bot2_points / (len(df) - len(df[df["winner"] == "tie"])), len(df), len(df) - len(df[df["winner"] == "tie"])

bot1_points, bot2_points, total_games, games_without_ties = add_score_columns(get_df("NaiveBot", "PuctBot"))
print(bot1_points, bot2_points, total_games, games_without_ties)

-0.02040816326530612 0.02040816326530612 50 49


In [27]:
def get_normalized_score(df):
    """
        create a tournament score for each bot
        win:  3 points
        tie:  1 point
        loss: 0 points

        returns (points: bot 1), (points: bot 2), (games played) 
    """

    bot1, bot2 = get_bots_playing(df)
    
    bot1_points = 0
    bot2_points = 0
    for index, row in df.iterrows():

        if row["winner"] == bot1:
            bot1_points += 1
        elif row["winner"] == bot2:
            bot2_points += 1

    return bot1_points / (len(df) - len(df[df["winner"] == "tie"])), bot2_points / (len(df) - len(df[df["winner"] == "tie"]))

bot1_score, bot2_score = add_normalized_score(get_df("AdderBoaCobra", "PUCTBot"))
print(bot1_score, bot2_score,)

0.6578947368421053 0.34210526315789475


In [32]:
def get_running_sum(df):
    """
        return df with running total of apples eaten
    """

    bot1, bot2 = get_bots_playing(df)
    
    bot1_points = 0
    bot2_points = 0
    bot1_apples_total = 0
    bot2_apples_total = 0
    df[f"{bot1} total"] = df["appleEaten0"].cumsum()
    df[f"{bot2} total"] = df["appleEaten1"].cumsum()


    
        
    return df[["iteration", "appleEaten0", f"{bot1} total", "appleEaten1",  f"{bot2} total"]]
    

total_apples_eaten_df = get_running_sum(adder_rollout_df)
# total_apples_eaten_df["iter"]
total_apples_eaten_df

NameError: name 'adder_rollout_df' is not defined

In [33]:
bot1, bot2 = get_bots_playing(adder_rollout_df)
total1 = f"{bot1} total"
total2 = f"{bot2} total"

total_apples_eaten_df.plot(kind="line", x="iteration", y=[total1, total2])

NameError: name 'adder_rollout_df' is not defined

## 1. Adder vs Rollout 

In [41]:
# bots playing
def get_tournament_statistics(df):
    bot1, bot2 = get_bots_playing(df)
    # average apples eaten:
    tournament_statistics_df = df.groupby("winner").agg(aggregations)
    bot1_points, bot2_points, total_games, games_without_ties = add_score_columns(df)
    tournament_statistics_df.loc[bot1 ,("Points (W=1 L=-1)")] = bot1_points
    tournament_statistics_df.loc[bot2 ,("Points (W=1 L=-1)")] = bot2_points

    tournament_statistics_df.loc[bot1 ,("Score", "Ties")] = tournament_statistics_df.loc[bot1,("winner", "count")] / total_games
    tournament_statistics_df.loc[bot1 ,("Score", "Without Ties")] = tournament_statistics_df.loc[bot1,("winner", "count")] / games_without_ties

    tournament_statistics_df.loc[bot2 ,("Score", "Ties")] = tournament_statistics_df.loc[bot2,("winner", "count")] / total_games
    tournament_statistics_df.loc[bot2 ,("Score", "Without Ties")] = tournament_statistics_df.loc[bot2,("winner", "count")] / games_without_ties


    # tournament_statistics_df.loc[bot1 ,("Score", "Ties")] = bot1_points / total_games
    # tournament_statistics_df.loc[bot1 ,("Score", "Without Ties")] = bot1_points / games_without_ties
    # tournament_statistics_df[""]
    try:
        tournament_statistics_df = tournament_statistics_df.drop(index="tie")
    except:
        # no ties
        pass
    return tournament_statistics_df
get_tournament_statistics(get_df("AdderBoaCobra", "RolloutBot"))

appleEaten0      appleEaten1        duration          winner  \
                     mean  sum        mean  sum       mean      sum  count   
winner                                                                       
AdderBoaCobra    5.655172  164    4.655172  135  28.251483  819.293     29   
RolloutBot       2.000000   10    3.200000   16  12.479000   62.395      5   

              Points (W=3 T=1) Score               
                                Ties Without Ties  
winner                                             
AdderBoaCobra         0.705882  0.58     0.852941  
RolloutBot           -0.705882  0.10     0.147059

## 2. Adder vs PUCT

In [42]:
get_tournament_statistics(get_df("AdderBoaCobra", "PUCTBot"))

appleEaten0      appleEaten1         duration           winner  \
                     mean  sum        mean  sum        mean       sum  count   
winner                                                                         
AdderBoaCobra    5.960000  149    4.280000  107  153.569720  3839.243     25   
PUCTBot          5.769231   75    7.384615   96  161.315769  2097.105     13   

              Points (W=3 T=1)     Score               
                                    Ties Without Ties  
winner                                                 
AdderBoaCobra         0.315789  0.595238     0.657895  
PUCTBot              -0.315789  0.309524     0.342105

## 3. Adder vs UCT

In [43]:
get_tournament_statistics(get_df("AdderBoaCobra", "TheBot"))

appleEaten0      appleEaten1         duration           winner  \
                     mean  sum        mean  sum        mean       sum  count   
winner                                                                         
AdderBoaCobra    5.234043  246    2.702128  127  205.454085  9656.342     47   
TheBot           5.000000    5    1.000000    1  118.912000   118.912      1   

              Points (W=3 T=1) Score               
                                Ties Without Ties  
winner                                             
AdderBoaCobra         0.958333  0.94     0.979167  
TheBot               -0.958333  0.02     0.020833

## 4. Adder vs Naive

In [44]:
get_tournament_statistics(get_df("AdderBoaCobra", "NaiveBot"))

appleEaten0      appleEaten1         duration           winner  \
                     mean  sum        mean  sum        mean       sum  count   
winner                                                                         
AdderBoaCobra    4.913043  113    6.565217  151  153.507478  3530.672     23   
NaiveBot         5.050000  101    4.950000   99  102.108950  2042.179     20   

              Points (W=3 T=1) Score               
                                Ties Without Ties  
winner                                             
AdderBoaCobra         0.069767  0.46     0.534884  
NaiveBot             -0.069767  0.40     0.465116

In [45]:
get_df("AdderBoaCobra", "NaiveBot").mean()

/var/folders/5_/2z9ny5950t57nyvwn68j6pg00000gn/T/ipykernel_24018/869230222.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  get_df("AdderBoaCobra", "NaiveBot").mean()


iteration       24.50000
appleEaten0      4.96000
appleEaten1      5.68000
duration       131.53802
dtype: float64

## 5. UCT vs Rollout

In [46]:
get_tournament_statistics(get_df("UCT", "RolloutBot"))

appleEaten0     appleEaten1      duration           winner  \
                  mean sum        mean  sum     mean       sum  count   
winner                                                                  
RolloutBot    1.844444  83    4.888889  220  60.9606  2743.227     45   
TheBot        3.600000  18    1.600000    8  27.1598   135.799      5   

           Points (W=3 T=1) Score               
                             Ties Without Ties  
winner                                          
RolloutBot              0.8   0.9          0.9  
TheBot                 -0.8   0.1          0.1

## 6. PUCT vs UCT

In [47]:
get_tournament_statistics(get_df("PUCTBot", "TheBot"))

appleEaten0      appleEaten1        duration           winner  \
               mean  sum        mean sum        mean       sum  count   
winner                                                                  
PUCTBot     7.95122  326     2.02439  83  189.405146  7765.611     41   
TheBot      1.50000    3     4.00000   8  325.004000   650.008      2   

        Points (W=3 T=1)     Score               
                              Ties Without Ties  
winner                                           
PUCTBot         0.906977  0.931818     0.953488  
TheBot         -0.906977  0.045455     0.046512

## 7. PUCT vs Rollout

In [48]:
get_tournament_statistics(get_df("PUCTBot", "RolloutBot"))

appleEaten0      appleEaten1         duration           winner  \
                  mean  sum        mean  sum        mean       sum  count   
winner                                                                      
PUCTBot       7.190476  151    8.571429  180  124.030905  2604.649     21   
RolloutBot    5.208333  125   10.250000  246  124.820583  2995.694     24   

           Points (W=3 T=1) Score               
                             Ties Without Ties  
winner                                          
PUCTBot           -0.066667  0.42     0.466667  
RolloutBot         0.066667  0.48     0.533333

## 8. Naive vs PUCT

In [49]:
get_tournament_statistics(get_df("NaiveBot", "PUCTBot"))

appleEaten0     appleEaten1       duration           winner  \
                mean sum        mean sum       mean       sum  count   
winner                                                                 
NaiveBot    1.916667  46    1.583333  38  65.160167  1563.844     24   
PUCTBot     1.560000  39    2.320000  58  77.938600  1948.465     25   

         Points (W=3 T=1) Score               
                           Ties Without Ties  
winner                                        
NaiveBot        -0.020408  0.48     0.489796  
PUCTBot          0.020408  0.50     0.510204

In [50]:
get_df("NaiveBot", "PUCTBot").sum()

iteration                                                   1225
winner         tieNaiveBotPUCTBotPUCTBotPUCTBotPUCTBotNaiveBo...
appleEaten0                                                   89
appleEaten1                                                  100
duration                                                3695.859
dtype: object

In [51]:
get_df("NaiveBot", "PUCTBot").mean()

/var/folders/5_/2z9ny5950t57nyvwn68j6pg00000gn/T/ipykernel_24018/1693548617.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  get_df("NaiveBot", "PUCTBot").mean()


iteration      24.50000
appleEaten0     1.78000
appleEaten1     2.00000
duration       73.91718
dtype: float64

## 9. Naive vs UCT(TheBot)

In [52]:
get_tournament_statistics(get_df("NaiveBot", "TheBot"))

appleEaten0     appleEaten1       duration           winner  \
                mean sum        mean sum       mean       sum  count   
winner                                                                 
NaiveBot    2.482759  72    0.931034  27  76.253000  2211.337     29   
TheBot      2.238095  47    1.095238  23  71.471857  1500.909     21   

         Points (W=3 T=1) Score               
                           Ties Without Ties  
winner                                        
NaiveBot             0.16  0.58         0.58  
TheBot              -0.16  0.42         0.42

## 10. Naive vs Rollout

In [53]:
get_tournament_statistics(get_df("NaiveBot", "RolloutBot"))

appleEaten0      appleEaten1        duration           winner  \
                  mean  sum        mean  sum       mean       sum  count   
winner                                                                     
NaiveBot      2.783784  103    4.216216  156  46.740135  1729.385     37   
RolloutBot    4.727273   52    9.727273  107  96.536273  1061.899     11   

           Points (W=3 T=1) Score               
                             Ties Without Ties  
winner                                          
NaiveBot           0.541667  0.74     0.770833  
RolloutBot        -0.541667  0.22     0.229167

In [54]:
bots = ["NaiveBot", "RolloutBot", "TheBot", "PUCTBot", "UCT", "RolloutBot"]

score_matrix = np.zeros((len(bots), len(bots)))

for i, bot1 in zip(range(len(bots)), (bots)):
    for j, bot2 in zip(range(len(bots)), (bots)):
        if i == j:
            continue
        try:
             get_normalized_score(get_df(bot1, bot2))
        except:
            score_matrix[i][j] = get_normalized_score(get_df(bot1, bot2))


FileNotFoundError: [Errno 2] No such file or directory: './NaiveBot_vs_UCT.csv'